In [2]:
!pip install google-api-python-client
!pip install anthropic

from googleapiclient.discovery import build
import concurrent.futures
import ast
import os
import re
import anthropic
import requests
import json

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Replace with your Anthropic API key
GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID

client = anthropic.Anthropic()

Per 1 million tokens:
    Input / Output




claude-3-opus-20240229      $15.00 / $75.00	

claude-3-sonnet-20240229    $3.00 / $15.00	

claude-3-haiku-20240307     $0.25 / $1.25

In [7]:
def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

def generate_text(prompt, model="claude-3-haiku-20240307", max_tokens=4000, temperature=0.3):
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "system": "You are a world-class researcher. Analyze the given information and generate a brief, yet comprehensive and well-structured report.",
        "messages": [{"role": "user", "content": prompt}],
    }
    try:
        response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        response_json = response.json()
        print(response_json)
        response_text = response_json['content'][0]['text']
        print(remove_first_line(response_text.strip()))
        return remove_first_line(response_text.strip())
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")

def search_web(search_term, api_key, cse_id):
    service = build("customsearch", "v1", developerKey=api_key)
    result = service.cse().list(q=search_term, cx=cse_id).execute()
    # Parse the JSON data
    parsed_data = json.loads(json.dumps(result))
    # Extract relevant text from the items
    relevant_text = []
    for item in parsed_data['items']:
        title = item['title']
        snippet = item.get('snippet', '')  # Use get() to handle missing 'snippet' key
        relevant_text.append(f"Title: {title}\nSnippet: {snippet}\n")
    # Print the relevant text
    print("\n".join(relevant_text))
    return relevant_text

def generate_subtopic_report(research_topic, subtopic):
    search_data = []
    all_queries = []
    search_cache = {}
    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"Generate 3 or 4 search queries to gather information on the subtopic '{subtopic}' of the research topic '{research_topic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
 
    initial_queries_response = generate_text(initial_queries_prompt)
    if initial_queries_response.startswith('[') and ']' in initial_queries_response:
        try:
            initial_queries = ast.literal_eval(initial_queries_response)
        except SyntaxError:
            print("Error: Invalid search query format. Skipping initial queries.")
            initial_queries = []
    else:
        print("Error: Search query format not found. Skipping initial queries.")
        initial_queries = []
   
    print(initial_queries)
    all_queries.extend(initial_queries)
    def search_and_cache(query):
        if query in search_cache:
            return search_cache[query]
        else:
            search_results = search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)
            search_cache[query] = search_results
            return search_results
    with concurrent.futures.ThreadPoolExecutor() as executor:
        search_results = list(executor.map(search_and_cache, initial_queries))
        search_data.extend(search_results)
 
    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt = f"When writing your report, make it incredibly detailed, thorough, specific, and well-structured. Use Markdown for formatting. Analyze the following search data and generate a comprehensive report on the subtopic '{subtopic}' of the research topic '{research_topic}':\n\n{str(search_data)}"
    report = generate_text(report_prompt, max_tokens=4000)
    print(f"Analyzing report and generating additional searches for subtopic: {subtopic}...")
    analysis_prompt = f"Analyze the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' and identify areas that need more detail or further information:\n\n{report}\n\n---\n\nHere are all the search queries you have used so far for this subtopic:\n\n{str(all_queries)}\n\n---\n\nGenerate 2 or 3 new and unique search queries to fill in the gaps and provide more detail on the identified areas. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
   
    additional_queries_response = generate_text(analysis_prompt)
    if additional_queries_response.startswith('[') and ']' in additional_queries_response:
        try:
            additional_queries = ast.literal_eval(additional_queries_response)
        except SyntaxError:
            print("Error: Invalid search query format. Skipping additional queries.")
            additional_queries = []
    else:
        print("Error: Search query format not found. Skipping additional queries.")
        additional_queries = []
   
    all_queries.extend(additional_queries)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        additional_search_results = list(executor.map(search_and_cache, additional_queries))
        search_data.extend(additional_search_results)
    print(f"Updating report with additional information for subtopic: {subtopic}...")
    update_prompt = f"Update the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' by incorporating the new information from the additional searches. Keep all existing information... only add new information:\n\n{report}\n\n---\n\nAdditional search data:\n\n{str(additional_search_results)}\n\n---\n\nGenerate an updated report that includes the new information and provides more detail in the identified areas. Use Markdown for formatting."
    report = generate_text(update_prompt, max_tokens=4000)
    print(f"Final report generated for subtopic: {subtopic}!")
    return report

def generate_comprehensive_report(research_topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt = f"Generate a comprehensive report on the research topic '{research_topic}' by combining the following reports on various subtopics:\n\n{subtopic_reports}\n\n---\n\nEnsure that the final report uses markdown formatting, and is a well-structured overview which covers all the key aspects of the subtopics. Make sure that it includes all key information in each of the reports, but in a better structured, well-formatted manner. Nothing -- absolutely nothing, should be left out. The final report should contain a SINGLE '## Conclusion' at the end of the full comprehensive report. If you forget to include ANY key information from any of the previous reports, you will face the consequences. Include a table of contents, with '## Conclusion' being the FINAL section of the report. Leave nothing out. Use Markdown for formatting."
    comprehensive_report = generate_text(comprehensive_report_prompt, model="claude-3-sonnet-20240229", max_tokens=4000)
    
    # Check if the report is cut short and generate additional content if needed
    while "## Conclusion" not in comprehensive_report:
        print("Report is cut short. Generating additional content...")
        continuation_prompt = f"The comprehensive report is cut short. Please continue generating the report from where it left off. Here's the current report:\n\n{comprehensive_report}\n\n---\n\nContinue the report, ensuring that all key information from the subtopic reports is included and that the report is well-structured and comprehensive. Use Markdown for formatting."
        additional_content = generate_text(continuation_prompt, model="claude-3-sonnet-20240229", max_tokens=4000)
        comprehensive_report += "\n\n" + additional_content
    
    print("Comprehensive report generated!")
    return comprehensive_report

# User input
research_topic = input("Enter the research topic: ")

# Generate subtopic checklist
subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist in a Python-parseable list. Return nothing but the list. Do so in one line. Maximum five sub-topics. Start your response with [\""
subtopic_checklist = ast.literal_eval('[' + generate_text(subtopic_checklist_prompt).split('[')[1])
print(f"Subtopic Checklist: {subtopic_checklist}")

# Generate reports for each subtopic
subtopic_reports = []
for subtopic in subtopic_checklist:
    subtopic_report = generate_subtopic_report(research_topic, subtopic)
    subtopic_reports.append(subtopic_report)

# Combine subtopic reports into a comprehensive report
comprehensive_report = generate_comprehensive_report(research_topic, "\n\n".join(subtopic_reports))

# Save the comprehensive report to a file
with open("comprehensive_report.txt", "w") as file:
    file.write(comprehensive_report)
print("Comprehensive report saved as 'comprehensive_report.txt'.")

{'id': 'msg_01DQh8QeLf2DR6cahsyNotM1', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': '["company history", "product portfolio", "marketing and branding strategies", "sustainability and social responsibility initiatives", "financial performance and growth"]'}], 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 93, 'output_tokens': 31}}
["company history", "product portfolio", "marketing and branding strategies", "sustainability and social responsibility initiatives", "financial performance and growth"]
Subtopic Checklist: ['company history', 'product portfolio', 'marketing and branding strategies', 'sustainability and social responsibility initiatives', 'financial performance and growth']
Generating initial search queries for subtopic: company history...
{'id': 'msg_01GsxYXiBKc9pMsCrTKSup1q', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': '["Ben & Jerry\'s Icecream c

# Comprehensive Report on Ben & Jerry's Ice Cream

## Table of Contents

1. [Company History](#company-history)
2. [Product Portfolio](#product-portfolio)
3. [Marketing and Branding Strategies](#marketing-and-branding-strategies)
4. [Sustainability and Social Responsibility Initiatives](#sustainability-and-social-responsibility-initiatives)
5. [Financial Performance and Growth](#financial-performance-and-growth)
6. [Conclusion](#conclusion)

## Company History

### Executive Summary
Ben & Jerry's is an iconic American ice cream company with a rich and storied history. Founded in 1978 by childhood friends Ben Cohen and Jerry Greenfield, the company has grown from a small scoop shop in Burlington, Vermont to a global brand known for its innovative flavors and commitment to social and environmental causes.

### Origins and Early Years
- Ben Cohen and Jerry Greenfield met in seventh grade gym class on Long Island, New York and remained friends throughout their lives.
- In 1978, the two friends decided to start an ice cream business, initially considering opening a bagel shop before settling on ice cream due to the high cost of the necessary equipment for bagels.
- After completing a $5 correspondence course in ice cream making, Cohen and Greenfield opened their first ice cream scoop shop in a renovated gas station in Burlington, Vermont on May 5, 1978.
- The company's early years were marked by rapid growth, with Ben & Jerry's expanding to multiple scoop shops and beginning to sell pints of ice cream in grocery stores less than two years after opening.

### Expansion and Diversification
- As Ben & Jerry's grew, the company began to experiment with unique and creative ice cream flavors, setting it apart from traditional ice cream brands.
- In the 1980s, Ben & Jerry's expanded beyond Vermont, opening scoop shops in other parts of the Northeastern United States and eventually nationwide.
- The company also diversified its product line, introducing frozen yogurt, sorbet, and other frozen treats.
- Ben & Jerry's gained national recognition and a loyal following for its commitment to using high-quality, all-natural ingredients and its support for social and environmental causes.

### Acquisition by Unilever
- In 2000, Ben & Jerry's was acquired by the multinational consumer goods company Unilever, sparking concerns among fans about the company's ability to maintain its values and social mission under new ownership.
- However, Ben & Jerry's has continued to operate as a distinct brand within Unilever, maintaining its commitment to social responsibility and progressive values.
- The company has continued to innovate and expand its product line, while also remaining active in various social and environmental initiatives, such as advocating for causes like climate justice, human rights, and economic justice.
- Unilever has recognized the value of Ben & Jerry's social mission and has sought to preserve it, with the company's purpose-led brands, including Ben & Jerry's, outperforming the rest of Unilever's portfolio.

### Ongoing Commitment to Social and Environmental Causes
- Throughout its history, Ben & Jerry's has been a leader in corporate social responsibility, dedicating a significant portion of its pre-tax earnings to social and environmental initiatives.
- The company has advocated for a wide range of causes, including human rights, economic justice, climate change, and environmental protection, often using its platform to raise awareness and drive change.
- Ben & Jerry's has maintained an independent Board of Directors empowered to protect the company's brand equity and integrity, ensuring that its social mission remains a core part of its business model even after the Unilever acquisition.
- The company continues to innovate and diversify its product line while staying true to its values, demonstrating that businesses can be profitable and socially responsible at the same time.

## Product Portfolio

### Introduction
Ben & Jerry's offers a wide range of ice cream and frozen dessert products, including ice cream flavors, non-dairy products, and other frozen treats. The company is known for its innovative and creative flavor combinations, as well as its commitment to using high-quality, all-natural ingredients.

### Ice Cream Flavors
- Ben & Jerry's has an extensive portfolio of ice cream flavors, including both classic and innovative options. Some of their popular flavors include Americone Dream, Cherry Garcia, Chocolate Fudge Brownie, Cookie Dough, and Strawberry Cheesecake.
- The company also offers a range of "Certified Gluten-Free" flavors, catering to consumers with dietary restrictions.
- In addition to pints, Ben & Jerry's sells ice cream in mini cups and other packaging formats.
- The company's "Flavor Gurus" are constantly working on creating new and unique flavor combinations, with the development process taking 18-24 months on average.

### Non-Dairy Products
- Ben & Jerry's has been expanding its non-dairy product portfolio in recent years, responding to the growing demand for plant-based alternatives.
- The company's non-dairy line is made with almond milk or sunflower butter and includes flavors such as Chocolate Fudge Brownie, Chunky Monkey, and Peanut Butter Cup.
- According to the information provided, Ben & Jerry's non-dairy products now make up 25% of its global product portfolio.
- The company has been working on developing a new, improved recipe for its non-dairy products, aiming to provide a smoother and creamier texture.

### Other Products
- In addition to ice cream, Ben & Jerry's offers other frozen dessert products, such as frozen yogurt and sorbet.
- The company has also introduced "Doggie Desserts," a line of frozen treats specifically formulated for pets.

### Product Development and Innovation
- Ben & Jerry's is known for its innovative approach to product development, with a team of "flavor gurus" constantly working on creating new and unique flavor combinations.
- The company has a strong focus on using high-quality, all-natural ingredients in its products, which is a key part of its brand identity.
- Ben & Jerry's has also been responsive to consumer trends and preferences, expanding its non-dairy offerings to cater to the growing demand for plant-based options.
- The company's product development process involves extensive research and testing, with the team working to ensure that new flavors and products meet the high standards of quality and taste that Ben & Jerry's is known for.

### Sustainable Packaging and Environmental Efforts
- Ben & Jerry's is committed to reducing the environmental impact of its packaging and operations.
- The company has partnered with the Canopy Pack4Good Initiative to explore more sustainable packaging options, including the use of recycled and renewable materials.
- Ben & Jerry's has also been working to improve the recyclability and compostability of its packaging, with the goal of making all of its packaging reusable, recyclable, or compostable by 2025.
- In addition to its packaging efforts, Ben & Jerry's has a strong focus on sustainability throughout its supply chain, including sourcing ingredients from sustainable farms and implementing renewable energy initiatives at its production facilities.

## Marketing and Branding Strategies

### Brand Identity and Storytelling
- Ben & Jerry's has developed a strong and recognizable brand identity, which is central to their marketing strategy.
- The brand is known for its commitment to social and environmental causes, which is reflected in its branding and marketing campaigns.
- Ben & Jerry's uses storytelling to connect with its customers and share the brand's values and mission. This includes highlighting the brand's support for various social and environmental causes, such as 1% for Peace, opposing artificial growth hormones, and supporting sustainable agriculture.
- The brand's unique and creative ice cream flavors are an integral part of its brand identity and marketing efforts, with new flavors often being launched to generate excitement and engagement.

### Marketing Channels and Strategies
- Ben & Jerry's utilizes a variety of marketing channels, including social media, content marketing, and in-store promotions, to reach and engage with its target audience.
- On social media, the brand focuses on creating engaging content, such as picture challenges and teasers for new product launches, to build brand awareness and loyalty. The brand's social media strategy also emphasizes its commitment to social and environmental causes, further reinforcing its brand identity.
- The brand's content marketing strategy involves aligning its marketing efforts with its social and environmental initiatives, such as creating content that highlights the brand's support for various causes or its commitment to sustainable practices.
- Ben & Jerry's also leverages in-store demonstrations and giveaways to promote its products and connect with customers, providing them with a hands-on experience with the brand's offerings.

### Targeting and Segmentation
- Ben & Jerry's targets customers who have unique flavor preferences and a taste for high-quality, premium ice cream. This includes a focus on younger, urban consumers (aged 20-35) who are part of the AB social class and have an interest in original and socially conscious products.
- The brand's marketing efforts are tailored to this target audience, focusing on the premium nature of its products and the brand's commitment to social and environmental causes. This includes highlighting the brand's use of high-quality ingredients, its support for sustainable agriculture, and its involvement in various social and environmental initiatives.
- Ben & Jerry's has also leveraged market segmentation to tailor its offerings and marketing strategies to different regions and demographics, such as creating unique flavors for specific markets or adjusting its messaging to resonate with local audiences.

## Sustainability and Social Responsibility Initiatives

### Sustainability Initiatives

#### Climate Impact and Reduction Efforts
- Ben & Jerry's has conducted extensive research on the carbon footprint of its ice cream production and is actively working to reduce its environmental impact.
- The company has set ambitious goals to reduce greenhouse gas emissions, aiming to use 100% renewable energy in its company-owned facilities by 2025.
- Ben & Jerry's has implemented various strategies to achieve these targets, such as investing in renewable energy sources like wind and solar, improving energy efficiency, and exploring innovative technologies like blockchain.
- The company has also collaborated with organizations like WWF to raise awareness and educate the public on the impacts of climate change, and has participated in initiatives like the Ready for 100 campaign to promote the transition to 100% clean energy in the U.S.

#### Sustainable Packaging
- Recognizing the importance of reducing waste, Ben & Jerry's has committed to making all of its packaging reusable, compostable, or recyclable by 2025.
- The company has been exploring and implementing various sustainable packaging solutions, such as using renewable and recycled materials, to minimize its environmental footprint.
- In 2021, Ben & Jerry's transitioned to a paper wrapper for its ice cream containers in Europe, making them more widely recyclable.

### Social Responsibility Initiatives

#### Values-Driven Approach
- Ben & Jerry's has a strong social mission that is deeply embedded in its core values and business practices, with a focus on issues like peace, human rights, and environmental justice.
- The company has advocated for various social and political causes, using its platform and resources to raise awareness and drive change.

#### Ethical Sourcing and Supplier Relationships
- Ben & Jerry's has implemented ethical sourcing practices, such as the Milk with Dignity program, to ensure fair treatment and working conditions for its suppliers and their employees.
- The company has also been recognized for its commitment to social responsibility, earning B Corp certification and high scores on social impact assessments.
- In 2022, Ben & Jerry's announced a transition to a sustainable, responsible, and ethical model for sourcing its dairy products, building on its long-standing commitment to Fairtrade Certified ingredients.

#### Community Engagement and Philanthropy
- Ben & Jerry's has a long history of supporting and engaging with local communities through various initiatives, including the Ben & Jerry's Foundation, which donates a portion of the company's pre-tax earnings to social causes.
- The company has also collaborated with non-profit organizations and advocacy groups to address social and environmental issues, such as the Ready for 100 campaign with the Sierra Club.
- Ben & Jerry's employees are actively involved in community initiatives, contributing their time and skills to support local organizations and causes.

## Financial Performance and Growth

### Financial Performance
- Ben & Jerry's annual revenue is reported to be $450.0 million, according to data from Zippia.
- The company has experienced consistent growth, with a compound annual growth rate (CAGR) of 9.2% between 2011 and 2016, as reported by Statista.
- However, in recent years, the financial performance of Ben & Jerry's has faced some challenges, with Unilever reporting that the ice cream segment was the company's slowest-growing segment, with only a 5.6% underlying sales growth.
- The extreme heat has been cited as a factor that has hurt ice cream sales, according to Unilever's earnings report.
- Ben & Jerry's holds a market share of 8.7% in the United States, based on sales of $477.1 million, according to Statista.
- Unilever expects Ben & Jerry's to account for about 3% of the world's ice cream market, with the brand's sales growing 9% in the previous year, outpacing the overall market.
- In the UK, Ben & Jerry's remains the dominant player, with a market share of 29% as of recently, according to Mintel.
- The US ice cream industry has a market size of $10.6 billion, and Ben & Jerry's is the 10th largest ice cream company in the United States, with 2022 sales of $910.68 million, as reported by Zippia.

### Growth Strategies
- Ben & Jerry's has a strong focus on social and environmental responsibility, which has been a key driver of its growth and brand reputation.
- The company has advocated for various causes, such as opposing the use of Recombinant Bovine Growth Hormone (rBGH) in dairy production, which has helped to differentiate its brand and appeal to socially conscious consumers.
- Ben & Jerry's has also explored opportunities for expansion, such as the launch of a new protein-based ice cream line, as reported in marketing databases.
- However, the company's relationship with its parent company, Unilever, has faced some challenges, with legal disputes and disagreements over the direction of the brand.
- Unilever has outlined a three-pronged plan to capture market share in the growing premium ice cream segment, with Ben & Jerry's and Magnum representing 35-45% of the company's ice cream portfolio.
- Unilever is considering spinning off its ice cream business, including Ben & Jerry's, to boost profits, as the segment has faced flat revenues and lower profit margins compared to the company's overall performance.

## Conclusion

Ben & Jerry's Ice Cream has a rich and inspiring history, marked by the entrepreneurial spirit and social consciousness of its founders. From its humble beginnings in a Vermont gas station to its current status as a global ice cream brand, the company has remained true to its values and has become a model for how businesses can balance profitability with a commitment to making a positive impact on the world.

The company's product portfolio is diverse and well-rounded, offering a wide range of ice cream flavors, non-dairy options, and other frozen desserts. Ben & Jerry's is known for its innovative approach to product development, with a team of "flavor gurus" constantly working on creating new and unique flavor combinations. The company has also been responsive to consumer trends and preferences, expanding its non-dairy offerings to cater to the growing demand for plant-based options.

Ben & Jerry's has developed a comprehensive and effective marketing and branding strategy that has contributed to its success as a leading ice cream brand. By focusing on a strong brand identity, leveraging various marketing channels, and targeting a specific customer segment, Ben & Jerry's has been able to build a loyal customer base and maintain its position as a socially conscious and premium ice cream brand.

The company has consistently demonstrated its commitment to sustainability and social responsibility, making it a leader in the ice cream industry and a model for other businesses to follow. Ben & Jerry's has implemented various initiatives to reduce its environmental impact, such as setting ambitious goals to reduce greenhouse gas emissions and exploring sustainable packaging solutions. The company has also been a strong advocate for social and environmental causes, using its platform and resources to raise awareness and drive change.

Despite facing some financial challenges in recent years, Ben & Jerry's has maintained a significant market share in the United States, the UK, and globally. The company's strong focus on social and environmental responsibility has been a key driver of its growth and brand reputation. However, to maintain its competitive edge, Ben & Jerry's will need to continue to innovate and adapt to changing market conditions, while staying true to its core values and commitment to social and environmental responsibility.

Overall, Ben & Jerry's Ice Cream has a unique and inspiring story, and its commitment to social and environmental responsibility, combined with its innovative and high-quality products, has made it a beloved and respected brand worldwide.